In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.current_device()

0

In [4]:
torch.cuda.get_device_name(0)

'Tesla K80'

In [5]:
torch.cuda.memory_allocated()

0

In [6]:
torch.cuda.memory_cached()

/usr/local/lib/python3.7/dist-packages/torch/cuda/memory.py:386: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)


0

In [7]:
torch.cuda.get_device_name(0)

'Tesla K80'

In [8]:
a = torch.FloatTensor([1.0,2.0])
a

tensor([1., 2.])

In [9]:
a.device

device(type='cpu')

In [10]:
a = torch.FloatTensor([1.0,2.0]).cuda()
a.device

device(type='cuda', index=0)

In [11]:
torch.cuda.memory_allocated()

512

In [12]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from torchvision.utils import make_grid
from torchvision import datasets

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [30]:
class Model(nn.Module):

    def __init__(self,in_features = 4, h1 =8, h2 = 9,out_features = 3):
        super().__init__()
        self.fc1 = nn.Linear(in_features , h1)
        self.fc2 = nn.Linear(h1, h2)
        self.out = nn.Linear(h2,out_features)

    def forward(self, x):

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [49]:
torch.manual_seed(32)
model = Model()
model

Model(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=9, bias=True)
  (out): Linear(in_features=9, out_features=3, bias=True)
)

In [50]:
#next(model.parameters()).is_cuda

In [51]:
gpumodel = model.cuda()
gpumodel

Model(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=9, bias=True)
  (out): Linear(in_features=9, out_features=3, bias=True)
)

In [52]:
#next(gpumodel.parameters()).is_cuda

In [53]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
path = "/content/drive/MyDrive/PyTorch/Data/iris.csv"
df = pd.read_csv(path)

X = df.drop("target", axis = 1).values
y = df["target"].values

In [55]:
from sklearn.model_selection import train_test_split

X_train,X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 33)


In [56]:
X_train = torch.FloatTensor(X_train).cuda()
X_test = torch.FloatTensor(X_test).cuda()

y_train = torch.LongTensor(y_train).cuda()
y_test = torch.LongTensor(y_test).cuda()

In [57]:
trainloader = DataLoader(X_train, batch_size = 60 , shuffle = True, pin_memory=True)
testloader = DataLoader(X_test, batch_size = 60 , shuffle = False, pin_memory=True)

In [58]:
gpumodel

Model(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=9, bias=True)
  (out): Linear(in_features=9, out_features=3, bias=True)
)

In [59]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gpumodel.parameters(), lr=0.01)

In [60]:
import time
epochs = 100
losses = []
start = time.time()
for i in range(epochs):
    i+=1
    y_pred = gpumodel.forward(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss)
    
    # a neat trick to save screen space:
    if i%10 == 1:
        print(f'epoch: {i:2}  loss: {loss.item():10.8f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print(f'TOTAL TRAINING TIME: {time.time()-start}')

epoch:  1  loss: 1.15071130
epoch: 11  loss: 0.93773133
epoch: 21  loss: 0.77982587
epoch: 31  loss: 0.60994011
epoch: 41  loss: 0.40079933
epoch: 51  loss: 0.25436324
epoch: 61  loss: 0.15053055
epoch: 71  loss: 0.10086948
epoch: 81  loss: 0.08128319
epoch: 91  loss: 0.07231428
TOTAL TRAINING TIME: 0.24321913719177246
